In [2]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [3]:
maps = pd.read_csv("../Data/Raw/google_maps.csv")
play = pd.read_csv("../Data/Raw/google_play.csv")
app = pd.read_csv("../Data/Raw/app_store.csv")

### Let's Check Info

In [4]:
maps.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130 entries, 0 to 129
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   reviewer  130 non-null    object
 1   rating    130 non-null    int64 
 2   date      130 non-null    object
 3   text      52 non-null     object
 4   response  1 non-null      object
 5   company   130 non-null    object
dtypes: int64(1), object(5)
memory usage: 6.2+ KB


In [5]:
play.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 106 entries, 0 to 105
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   reviewId              106 non-null    object
 1   userName              106 non-null    object
 2   userImage             106 non-null    object
 3   content               106 non-null    object
 4   score                 106 non-null    int64 
 5   thumbsUpCount         106 non-null    int64 
 6   reviewCreatedVersion  93 non-null     object
 7   at                    106 non-null    object
 8   replyContent          26 non-null     object
 9   repliedAt             26 non-null     object
 10  appVersion            93 non-null     object
 11  company               106 non-null    object
dtypes: int64(2), object(10)
memory usage: 10.1+ KB


In [6]:
app.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 914 entries, 0 to 913
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   rating    914 non-null    int64 
 1   review    914 non-null    object
 2   userName  914 non-null    object
 3   title     914 non-null    object
 4   date      914 non-null    object
 5   company   914 non-null    object
dtypes: int64(1), object(5)
memory usage: 43.0+ KB


<font size=3> We need to standardize these three review datasets by aligning their columns, handling missing values, and converting data types. This involves mapping similar fields (like "score" and "rating"), cleaning text, formatting dates, and filling or flagging empty entries.
</font>

In [7]:
def safe_prepare(df, rename_map, source):
    df = df.copy()
    df = df.rename(columns=rename_map)
    df['source'] = source

    return df

In [8]:
app_map = {
    'review': 'text',
    'userName': 'reviewer',
    }
play_map = {
    'reviewId': 'review_id',
    'userName': 'reviewer',
    'content': 'text',
    'score': 'rating',
    'replyContent': 'response',
    'at': 'date',
    }

play = safe_prepare(play,play_map,'google_play')
app = safe_prepare(app,app_map,'app_store')

In [9]:
maps['source'] = 'google_maps'

In [10]:
maps.date.unique()

array(['3 days ago', '5 days ago', 'a week ago', '3 weeks ago',
       '4 weeks ago', 'a month ago', '2 months ago', '18 hours ago',
       '2 weeks ago', '3 months ago', '4 months ago', '5 months ago',
       '6 months ago', '7 months ago', '8 months ago', '9 months ago',
       '10 months ago', '11 months ago', 'a year ago'], dtype=object)

In [12]:
data_str = ['2 months ago', 'a month ago', 'a year ago', '5 months ago', 
'6 months ago', '3 months ago', '4 months ago', '7 months ago',
'3 weeks ago', '11 months ago', '2 weeks ago', '5 days ago',
'8 months ago', '10 months ago', '4 weeks ago', 'a week ago',
'9 months ago', '18 hours ago', '3 days ago']


def relative_to_date(rel_str):
    now = datetime.now()
    rel_str = rel_str.lower()
    
    if 'hour' in rel_str:
        hours = int(rel_str.split()[0]) if rel_str[0].isdigit() else 1
        return now - timedelta(hours=hours)
    elif 'day' in rel_str:
        days = int(rel_str.split()[0]) if rel_str[0].isdigit() else 1
        return now - timedelta(days=days)
    elif 'week' in rel_str:
        weeks = int(rel_str.split()[0]) if rel_str[0].isdigit() else 1
        return now - timedelta(weeks=weeks)
    elif 'month' in rel_str:
        months = int(rel_str.split()[0]) if rel_str[0].isdigit() else 1
        return now - timedelta(days=months*30)  # Approximate
    elif 'year' in rel_str:
        years = int(rel_str.split()[0]) if rel_str[0].isdigit() else 1
        return now - timedelta(days=years*365)  # Approximate
    else:
        return now

relative_to_date('1 day ago')

datetime.datetime(2025, 4, 28, 16, 38, 8, 92793)

In [13]:
maps['date'] = maps['date'].apply(relative_to_date)
maps['date'] = maps['date'].dt.strftime('%Y-%m-%d')

In [14]:
play['date'] = pd.to_datetime(play['date'], errors='coerce')
app['date'] = pd.to_datetime(app['date'],errors='coerce')
play['date'] = play['date'].dt.strftime('%Y-%m-%d')
app['date'] = app['date'].dt.strftime('%Y-%m-%d')

In [15]:
## we missing the response column lets add it as NaN
app['response'] = np.NaN

In [16]:
common = ['reviewer', 'rating', 'date', 'text', 'source','company','response']

df = pd.concat([maps[common],app[common],play[common]],ignore_index=False)

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1150 entries, 0 to 105
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   reviewer  1150 non-null   object
 1   rating    1150 non-null   int64 
 2   date      1150 non-null   object
 3   text      1072 non-null   object
 4   source    1150 non-null   object
 5   company   1150 non-null   object
 6   response  27 non-null     object
dtypes: int64(1), object(6)
memory usage: 71.9+ KB


#### Data Statistic

In [18]:
# data size and  shape
print(f'The Data has {df.shape[0]} records and {df.shape[1]} features')

The Data has 1150 records and 7 features


In [19]:
# checking for data statistic
df.describe()

,rating
count,1150.000000
mean,3.243478
std,1.870905
min,1.000000
25%,1.000000
50%,4.000000
75%,5.000000
max,5.000000


<font size=3>The rating data shows an average score of 3.24 (out of 5) with significant variability (standard deviation of 1.87), indicating diverse customer experiences. While half of the ratings are 4 or higher (median = 4), a quarter of reviews give the lowest score (1-star), suggesting polarized feedback. The full range (1-5 stars) is represented, with 5-stars being the most common high rating (75th percentile). This distribution implies that while many customers are satisfied, a substantial portion had strongly negative experiences</font>

In [20]:
df.isna().sum()

reviewer       0
rating         0
date           0
text          78
source         0
company        0
response    1123
dtype: int64

<font size=3>For missing text fields, we'll infer sentiment from rating (e.g., 5 stars = "Positive"). Empty response fields can stay as NaN, correctly indicating no company reply. This keeps data meaningful while reflecting reality.</font>

In [21]:
df.loc[df['text'].isna() & (df['rating'] >= 4), 'text'] = "Positive experience"
df.loc[df['text'].isna() & (df['rating'] <= 2), 'text'] = "Negative experience"
df.loc[df['text'].isna() & (df['rating'] == 3), 'text'] = "Neutral  experience"

In [22]:
#we use true if there is  a response else false
df['response'] = df['response'].notna()

In [23]:
# there is no duplicates in our data 
df.duplicated().sum()

0

In [28]:
# we turn date from objet to datetime
df['date'] = pd.to_datetime(df['date'],errors='coerce')

In [30]:
df['date']

0     2025-04-26
1     2025-04-24
2     2025-04-22
3     2025-04-08
4     2025-04-01
         ...    
101   2025-04-28
102   2025-04-28
103   2025-04-28
104   2025-04-28
105   2025-04-28
Name: date, Length: 1150, dtype: datetime64[ns]

In [32]:
# save it
df.to_csv("../Data/Cleaned/cleaned.csv",index=False)